ابتدا داده ها را می خوانیم و آن را داخل متغییر می ریزیم 

In [4]:
import pandas as pd
data = pd.read_csv("covtype.csv") #reading the data

سپس داده ها را جدا کرده، به این صورت که ۵۴ ستون اول را در یک متغییر و ستون آخر را در یک متغییر میریزیم

In [5]:
X = data.iloc[:, :-1] #fist 54 columns with all rows
y = data.iloc[:, -1] #last column with all rows (the answer column)

حال ۱۰ ستون اول داده ها را که نرمال نیستند، نرمالایز می کنیم

In [6]:
from sklearn import preprocessing
norm_X = X.copy()
norm_X.iloc[:, :10] = preprocessing.normalize(X.iloc[:, 0:10]) #normalizing first 10 columns of X

سپس داده ها را یک بر می زنیم و به صورت ۳۰-۷۰ آنها را بین داده های یادگیری و تست، تقسیم می کنیم

In [7]:
from sklearn.model_selection import train_test_split
nX_train, nX_test, y_train, y_test = train_test_split(norm_X, y, test_size = 0.3, stratify = y)

حال ۱۲ حالت را برای الگوریتم امتحان می کنیم:

n_estimators = [80, 100, 120]

max_feature = [5, 7]

criterion = ["gini", "entropy"]

در اینجا با استفاده از داده های  یادگیری، این حالت ها را با الگوریتم روش جنگل تصادفی و محاسبه ارزش ها به وسیله اعتبار سنجی متقابل ۵ برابر، محاسبه می کنیم.
( به خاطر به هم نریختن جمله ها مجبور شدم از اصطلاحات فارسی استفاده کنم )

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
est = [80, 100, 120]
max_feature = [5, 7]
cri = ["gini", "entropy"]
scores = np.zeros(12)
i=0;
for a in est:
    for b in max_feature:
        for c in cri:
            print(i)
            RFC = RandomForestClassifier(n_estimators = a, max_features = b, criterion = c)
            score = cross_val_score(RFC, nX_train, y_train, cv = 5)
            scores[i] = np.mean(score)
            i = i+1

0
1
2
3
4
5
6
7
8
9
10
11


حال نمره ها را خروجی می دهیم تا بررسی کنیم که کدام حالت برای این داده ها با این الگوریتم مناسب تر است.

In [7]:
print(scores)

[0.9468759  0.94751518 0.94791104 0.94930762 0.94671362 0.94784219
 0.94821346 0.94948956 0.94710456 0.94824789 0.94867571 0.94974528]


به دلیل طولانی بودن زمان ران شدن سلول کد شماره ۵، نمودار خروجی ندارد اما کد زمیمه می شود

و مشخصا همه ارزش ها نزدیک هم هستند و حالت آخر ارزش بیشتری دارد.

In [15]:
import matplotlib.pyplot as plt
plt.plot(scores, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
plt.show()

NameError: name 'scores' is not defined

طبق نتایج بدست آمده، همه حالت ها تقریبا نزدیک هم هستند اما بهترین حالت برابر است با:

n_estimators = 120, max_features = 7, criterion = "entropy"

n_estimators =تعداد داده ها

max_features = بیشترین تعداد ستون که می توان از داده های آنها استفاده کرد

criterion = معیاری برای برازش تقسیم بندی

In [9]:
import numpy as np
RFC = RandomForestClassifier(n_estimators = 20, max_features = 5, criterion = "gini")
score = cross_val_score(RFC, nX_train, y_train, cv = 5)
print(np.mean(score))

0.9399347893270928


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
RFC = RandomForestClassifier(n_estimators = 120, max_features = 7, criterion = "entropy")
score = cross_val_score(RFC, nX_train, y_train, cv = 5)

حال اعتبار حالت آخر را دوباره حساب کرده و خروجی می دهیم و آن را در

RFC

نگه می داریم

In [11]:
import numpy as np
print(np.mean(score))

0.9504927340210527


حال یک تابع به داده های یادگیری برازش می دهیم و سپس داده های تست را امتحان می کنیم. مشخص می شود که این تابع، ۹۵.۵۶ درصد، روی داده های تست به درستی کلاس بندی می کنید.

بنابراین:

accuracy = 0.9556

In [12]:
from sklearn.metrics import accuracy_score
RFC.fit(nX_train, y_train)
y_prediction = RFC.predict(nX_test)

acc = accuracy_score(y_prediction, y_test)
acc

0.9556923535891316

حال ماتریس در هم ریختگی را بدست آورده که درستی کلاس بندی داده های پیش بینی را نسبت به داده های تست ما بررسی می کند

In [13]:
from sklearn.metrics import confusion_matrix
per_metrics = confusion_matrix(y_test, y_prediction)
per_metrics

array([[60177,  3212,     1,     0,    17,     9,   136],
       [ 1979, 82628,   150,     4,   112,    95,    23],
       [    1,   157, 10314,    42,     7,   205,     0],
       [    0,     2,   108,   684,     0,    30,     0],
       [   53,   459,    42,     0,  2286,     8,     0],
       [    3,   205,   340,    18,     8,  4636,     0],
       [  240,    55,     0,     0,     2,     0,  5856]], dtype=int64)

مثلا در سطر اول و ستون اول، ۶۰۱۷۷ داده به درستی در کلاس اول قرار گرفته اند، ولی در سطر آخر این ستون، ۲۴۰ داده که باید در کلاس ۷ باشند، در کلاس اول قرار داده شده اند.

در کل هر سطر نشان دهنده کلاس واقعی و هر ستون نشان دهنده کلاس پیش بینی شده است.

حال در ماتریس زیر مقادیر

precision در سطر اول

recall در سطر دوم

f_measure در سطر سوم

و جمع هر سطر ماتریس در هم ریختگی بالا در سطر آخر آمده است.

در واقع سطر آخر بیان می کند که چند داده در هر کلاس داریم.

In [14]:
from sklearn.metrics import precision_recall_fscore_support
cost_sensitive_measures = precision_recall_fscore_support(y_test, y_prediction)
cost_sensitive_measures

(array([0.96355659, 0.95283563, 0.94148791, 0.9144385 , 0.93996711,
        0.93036323, 0.97356608]),
 array([0.94689388, 0.97219706, 0.96158866, 0.83009709, 0.80266854,
        0.88982726, 0.95173086]),
 array([0.95515257, 0.96241898, 0.95143213, 0.87022901, 0.86590909,
        0.90964387, 0.96252465]),
 array([63552, 84991, 10726,   824,  2848,  5210,  6153], dtype=int64))

در این ماتریس، مثلا برای کلاس اول

precision = 0.9635

که به این معنا است که ۰.۹۶۳۵ از داده هایی که در کلاس اول پیش بینی شده اند، واقعا مربوط به کلاس اول هستند.

کم ترین مقدار recall

برای کلاس پنجم است. در واقع بیان می کند که داده های کلاس پنجم به درستی پیش بینی نشده و ۲۰ درصد آنها داخل کلاس های دیگر پیش بینی شده اند.

حال داده های یادگیری با همین الگوریتم پیش بینی کرده و دقت پیش بینی را بدست می آوریم.

همانگونه که مشخص است و انتظار داشتیم، برازش داده های یادگیری را به درستی پیش بینی می کند و دقت آن ۱ است.

In [15]:
y_train_prediction = RFC.predict(nX_train)

acc = accuracy_score(y_train_prediction, y_train)
acc

1.0

حال ماتریس در هم ریختگی را برای داده های یادگیری می سازیم.

همانگونه که انتظار داریم همه کلاس ها به درستی پیش بینی شده اند.

از این ماتریس می توان متوجه شد که موقع بر زدن، چه مقدار از هر کلاس وارد داده یادگیری شده اند.

In [16]:
from sklearn.metrics import confusion_matrix
per_metrics = confusion_matrix(y_train, y_train_prediction)
per_metrics

array([[148288,      0,      0,      0,      0,      0,      0],
       [     0, 198310,      0,      0,      0,      0,      0],
       [     0,      0,  25028,      0,      0,      0,      0],
       [     0,      0,      0,   1923,      0,      0,      0],
       [     0,      0,      0,      0,   6645,      0,      0],
       [     0,      0,      0,      0,      0,  12157,      0],
       [     0,      0,      0,      0,      0,      0,  14357]],
      dtype=int64)

حال در ماتریس زیر

precision در سطر اول

recall در سطر دوم

f_measure در سطر سوم

و جمع هر سطر ماتریس در هم ریختگی بالا در سطر آخر آمده است

برای داده های یادگیری آمده است

در واقع سطر آخر بیان می کند که چند داده در هر کلاس داریم.

همانگونه که انتظار داشتیم همه این مقادیر برای همه سطر ها یک شدند، که به این معنی است که همه پیش بینی ها درست بودند.

In [17]:
from sklearn.metrics import precision_recall_fscore_support
cost_sensitive_measures = precision_recall_fscore_support(y_train, y_train_prediction)
cost_sensitive_measures

(array([1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1.]),
 array([148288, 198310,  25028,   1923,   6645,  12157,  14357],
       dtype=int64))

حال به بیان مفهوم مقادیر بالا برای هر کلاس می پردازیم:

accuracy:

این مقدار بیان می کند که از ۰ تا ۱، چه مقدار برازش و پیش بینی های ما درست بوده. در واقع نشان دهنده میزان درستی تابع بدست آمده براساس متغییر ها برای تشخیص کلاس هر داده است.

precision:

این مقدار بیان می کند که از ۰ تا ۱، چه مقدار از داده هایی که در آن کلاس پیش بینی شده اند، واقعا عضو آن کلاس هستند.

recall:

این مقدار بیان می کند که از ۰ تا ۱، چه مقدار از داده های متعلق به هر کلاس، به درستی داخل آن کلاس پیش بینی شده اند.